## 따릉이 대여량 예측
##### 1. 데이터 이용해서 따릉이 대여량 예측하는 regression 문제
##### 2. 평가 Metric : RMSE

In [284]:
# 필요한 라이브러리 불러오기

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
plt.style.use('seaborn-whitegrid')
import missingno

# 전처리 및 머신 러닝 알고리즘
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import StandardScaler

# 모델 튜닝 및 평가
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_predict
from sklearn import model_selection
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.linear_model import BayesianRidge
from xgboost.sklearn import XGBRegressor
from sklearn.ensemble import RandomForestRegressor


# 경고 제거 (판다스가 에러 메세지를 자주 만들어 내기 때문에 이를 일단 무시하도록 설정합니다.)
import sys
import warnings

import warnings
warnings.filterwarnings('ignore')

In [285]:
# Load Data as Dataframe
test = pd.read_csv('../../../input/ddareung/test.csv')
train = pd.read_csv('../../../input/ddareung/train.csv')
combine = [test, train]

In [286]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1459 entries, 0 to 1458
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      1459 non-null   int64  
 1   hour                    1459 non-null   int64  
 2   hour_bef_temperature    1457 non-null   float64
 3   hour_bef_precipitation  1457 non-null   float64
 4   hour_bef_windspeed      1450 non-null   float64
 5   hour_bef_humidity       1457 non-null   float64
 6   hour_bef_visibility     1457 non-null   float64
 7   hour_bef_ozone          1383 non-null   float64
 8   hour_bef_pm10           1369 non-null   float64
 9   hour_bef_pm2.5          1342 non-null   float64
 10  count                   1459 non-null   float64
dtypes: float64(9), int64(2)
memory usage: 125.5 KB


In [287]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 715 entries, 0 to 714
Data columns (total 10 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   id                      715 non-null    int64  
 1   hour                    715 non-null    int64  
 2   hour_bef_temperature    714 non-null    float64
 3   hour_bef_precipitation  714 non-null    float64
 4   hour_bef_windspeed      714 non-null    float64
 5   hour_bef_humidity       714 non-null    float64
 6   hour_bef_visibility     714 non-null    float64
 7   hour_bef_ozone          680 non-null    float64
 8   hour_bef_pm10           678 non-null    float64
 9   hour_bef_pm2.5          679 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 56.0 KB


In [288]:
train.head()
# id : 따릉이 고유 id
# hour : 시간
# temperature : 기온
# precipication : 1 if rained else 0
# windspeed : 풍속
# humidity : 습도
# visibility : 기상 상태에 따른 가시성(가시거리?)
# ozone : 오존 농도
# pm10 : 미세먼지 농도 나쁨
# pm2.5 : 미세먼지 농도 보통
# count : 시간에 따른 따릉이 대여 수

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
0,3,20,16.3,1.0,1.5,89.0,576.0,0.027,76.0,33.0,49.0
1,6,13,20.1,0.0,1.4,48.0,916.0,0.042,73.0,40.0,159.0
2,7,6,13.9,0.0,0.7,79.0,1382.0,0.033,32.0,19.0,26.0
3,8,23,8.1,0.0,2.7,54.0,946.0,0.040,75.0,64.0,57.0
4,9,18,29.5,0.0,4.8,7.0,2000.0,0.057,27.0,11.0,431.0


In [289]:
train.describe()

,id,hour,hour_bef_temperature,hour_bef_precipitation,hour_bef_windspeed,hour_bef_humidity,hour_bef_visibility,hour_bef_ozone,hour_bef_pm10,hour_bef_pm2.5,count
count,1459.000000,1459.000000,1457.000000,1457.000000,1450.000000,1457.000000,1457.000000,1383.000000,1369.000000,1342.000000,1459.000000
mean,1105.914325,11.493489,16.717433,0.031572,2.479034,52.231297,1405.216884,0.039149,57.168736,30.327124,108.563400
std,631.338681,6.922790,5.239150,0.174917,1.378265,20.370387,583.131708,0.019509,31.771019,14.713252,82.631733
min,3.000000,0.000000,3.100000,0.000000,0.000000,7.000000,78.000000,0.003000,9.000000,8.000000,1.000000
25%,555.500000,5.500000,12.800000,0.000000,1.400000,36.000000,879.000000,0.025500,36.000000,20.000000,37.000000
50%,1115.000000,11.000000,16.600000,0.000000,2.300000,51.000000,1577.000000,0.039000,51.000000,26.000000,96.000000
75%,1651.000000,17.500000,20.100000,0.000000,3.400000,69.000000,1994.000000,0.052000,69.000000,37.000000,150.000000
max,2179.000000,23.000000,30.000000,1.000000,8.000000,99.000000,2000.000000,0.125000,269.000000,90.000000,431.000000


In [290]:
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
            print('------------------------------------------------------')
            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return missing_col

missing_col = check_missing_col(train)
print(missing_col)

결측치가 있는 컬럼은: hour_bef_temperature입니다
해당 컬럼에 총 2개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_precipitation입니다
해당 컬럼에 총 2개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_windspeed입니다
해당 컬럼에 총 9개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_humidity입니다
해당 컬럼에 총 2개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_visibility입니다
해당 컬럼에 총 2개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_ozone입니다
해당 컬럼에 총 76개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_pm10입니다
해당 컬럼에 총 90개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_pm2.5입니다
해당 컬럼에 총 117개의 결측치가 존재합니다.
------------------------------------------------------
[['hour_bef_temperature', dtype('float64')], ['hour_bef_precipitation', dtype('floa

In [291]:
missing_col = check_missing_col(test)

결측치가 있는 컬럼은: hour_bef_temperature입니다
해당 컬럼에 총 1개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_precipitation입니다
해당 컬럼에 총 1개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_windspeed입니다
해당 컬럼에 총 1개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_humidity입니다
해당 컬럼에 총 1개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_visibility입니다
해당 컬럼에 총 1개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_ozone입니다
해당 컬럼에 총 35개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_pm10입니다
해당 컬럼에 총 37개의 결측치가 존재합니다.
------------------------------------------------------
결측치가 있는 컬럼은: hour_bef_pm2.5입니다
해당 컬럼에 총 36개의 결측치가 존재합니다.
------------------------------------------------------


#### NA 값을 abnormal 값으로 전처리한 결과

In [87]:
train = train.fillna(int(-1000))
test  = test.fillna(int(-1000))
train = train.drop('id',axis =1)
test  = test.drop('id',axis = 1)

In [88]:
train_x = train.drop(['count'],axis = 1)
train_y = train['count']

In [89]:
# RandomForestRegressor
# GradientBoostingRegressor
# BayesianRidge
# XGBRegressor
model=RandomForestRegressor(n_estimators=100)
model.fit(train_x,train_y)
pred = model.predict(test)

submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('./abnormal_na_value/NA_-1000/simple_rfr2.csv',index = False)

In [90]:
model=GradientBoostingRegressor(n_estimators=100)
model.fit(train_x,train_y)
pred = model.predict(test)

submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('./abnormal_na_value/NA_-1000/simple_gbr2.csv',index = False)

In [91]:
model=BayesianRidge()
model.fit(train_x,train_y)
pred = model.predict(test)

submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('./abnormal_na_value/NA_-1000/simple_baysianridge2.csv',index = False)

In [92]:
model=XGBRegressor()
model.fit(train_x,train_y)
pred = model.predict(test)

submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('./abnormal_na_value/NA_-1000/simple_xgb2.csv',index = False)

#### NA 값을 직관적인 사전 지식을 이용한 값으로 전처리한 결과

NA 값 전략
1. temperature : 전체 평균으로  --> ....?
2. precipitation : humidity > 80 이면 1 else 0
3. windspeed : 전체평균으로 --> 표준편차가 작음
4. humidity : precipitation 1 이면 humidity 80 else 20
--> if precipitation 과 humidity 가 둘다 NA 이면 그 row 는 Drop 하기
5. visibility : pm10 >90 이면, 200 으로
6. ozone : 전체 평균 값으로 --> 표준편차가 작음
7. pm10 : visibility 에 따라서 선형적으로 변하게 하기
8. pm2.5 : pm10 값에 1/2 

Quick Method 
--> drop NA row

In [292]:
# NA value 처리
train = train.drop('id',axis =1)
test  = test.drop('id',axis = 1)

In [293]:
train['hour_bef_temperature'].fillna(train['hour_bef_temperature'].dropna().mean(), inplace=True)
train['hour_bef_windspeed'].fillna(train['hour_bef_windspeed'].dropna().mean(),inplace=True)
train['hour_bef_ozone'].fillna(train['hour_bef_ozone'].dropna().mean(), inplace = True)

test['hour_bef_temperature'].fillna(test['hour_bef_temperature'].dropna().mean(), inplace=True)
test['hour_bef_windspeed'].fillna(test['hour_bef_windspeed'].dropna().mean(),inplace=True)
test['hour_bef_ozone'].fillna(test['hour_bef_ozone'].dropna().mean(), inplace = True)

In [294]:
null_datasets_train = pd.DataFrame()
null_datasets_test  = pd.DataFrame()

for dataset in train :
    null_datasets_train[dataset] = train[dataset].isnull()

for idx in range(len(train['hour'])):
    if (null_datasets_train['hour_bef_precipitation'][idx]) and (null_datasets_train['hour_bef_humidity'][idx]):
        if train['hour_bef_humidity'][idx] > 80.0:
            train['hour_bef_precipitation'][idx] = 1
        else :
            train['hour_bef_precipitation'][idx] = 0

for idx in range(len(train['hour'])):
    if (null_datasets_train['hour_bef_humidity'][idx]) :
        if (null_datasets_train['hour_bef_precipitation'][idx]):
            train['hour_bef_humidity'].fillna(train['hour_bef_humidity'].dropna().mean(), inplace=True) # 이경우 밖에 없었음
        else :
            if train['hour_bef_precipitation'][idx] == 1:
                train['hour_bef_humidity'][idx] = 80.0
            else :
                train['hour_bef_humidity'][idx] = 20.0

for idx in range(len(train['hour'])):
    if (null_datasets_train['hour_bef_visibility'][idx]) :
        if (null_datasets_train['hour_bef_pm10'][idx]):
            train['hour_bef_visibility'].fillna(train['hour_bef_visibility'].dropna().mean(), inplace=True)
        else :
            if train['hour_bef_pm10'][idx] > 90.0:
                train['hour_bef_visibility'][idx] = 200.0
            else :
                train['hour_bef_visibility'][idx] = 800.0
                
for idx in range(len(train['hour'])):
    if (null_datasets_train['hour_bef_pm10'][idx]) :
        if (null_datasets_train['hour_bef_visibility'][idx]):
            train['hour_bef_pm10'].fillna(train['hour_bef_pm10'].dropna().mean(), inplace=True)
        else :
            train['hour_bef_pm10'].interpolate()

for idx in range(len(train['hour'])):
    if (null_datasets_train['hour_bef_pm2.5'][idx]) :
        train['hour_bef_pm2.5'][idx] = train['hour_bef_pm10'][idx]/2

In [295]:
for dataset in test :
    null_datasets_test[dataset] = test[dataset].isnull()

for idx in range(len(test['hour'])):
    if (null_datasets_test['hour_bef_precipitation'][idx]) and (null_datasets_test['hour_bef_humidity'][idx]):
        if test['hour_bef_humidity'][idx] > 80.0:
            test['hour_bef_precipitation'][idx] = 1
        else :
            test['hour_bef_precipitation'][idx] = 0

for idx in range(len(test['hour'])):
    if (null_datasets_test['hour_bef_humidity'][idx]) :
        if (null_datasets_test['hour_bef_precipitation'][idx]):
            test['hour_bef_humidity'].fillna(test['hour_bef_humidity'].dropna().mean(), inplace=True) # 이경우 밖에 없었음
        else :
            if test['hour_bef_precipitation'][idx] == 1:
                test['hour_bef_humidity'][idx] = 80.0
            else :
                test['hour_bef_humidity'][idx] = 20.0

for idx in range(len(test['hour'])):
    if (null_datasets_test['hour_bef_visibility'][idx]) :
        if (null_datasets_test['hour_bef_pm10'][idx]):
            test['hour_bef_visibility'].fillna(test['hour_bef_visibility'].dropna().mean(), inplace=True)
        else :
            if test['hour_bef_pm10'][idx] > 90.0:
                test['hour_bef_visibility'][idx] = 200.0
            else :
                test['hour_bef_visibility'][idx] = 800.0
                
for idx in range(len(test['hour'])):
    if (null_datasets_test['hour_bef_pm10'][idx]) :
        if (null_datasets_test['hour_bef_visibility'][idx]):
            test['hour_bef_pm10'].fillna(test['hour_bef_pm10'].dropna().mean(), inplace=True)
        else :
            test['hour_bef_pm10'].interpolate()

for idx in range(len(test['hour'])):
    if (null_datasets_test['hour_bef_pm2.5'][idx]) :
        test['hour_bef_pm2.5'][idx] = test['hour_bef_pm10'][idx]/2

In [296]:
missing_col = check_missing_col(train)

결측치가 존재하지 않습니다


In [297]:
missing_col = check_missing_col(test)

결측치가 존재하지 않습니다


In [298]:
train_x = train.drop(['count'],axis = 1)
train_y = train['count']

In [299]:
# RandomForestRegressor
# GradientBoostingRegressor
# BayesianRidge
# XGBRegressor
model=RandomForestRegressor(n_estimators=100)
model.fit(train_x,train_y)
pred = model.predict(test)

submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('./use_prior_knowledge_for_NA/new_simple_rfr.csv',index = False)

In [302]:
model=GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01)
model.fit(train_x,train_y)
pred = model.predict(test)

submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('./use_prior_knowledge_for_NA/new_simple_gbr.csv',index = False)

In [ ]:
model=GradientBoostingRegressor(n_estimators=1000, learning_rate=0.01)
model.fit(train_x,train_y)
pred = model.predict(test)

submission = pd.read_csv('submission.csv')
submission['count'] = pred
submission.to_csv('./use_prior_knowledge_for_NA/new_simple_gbr.csv',index = False)